# Interpolation Functions for Circumstellar Planet Stability


The go-to source for determining whether a circumstellar planet (i.e., planet that orbits only one of two stars in a binary) has been an empirically determined fitting formula by Holman & Wiegert (1999).  In their study, the key assumptions are that the planet begins on a circular, coplanar orbit and does not have sufficient mass to influence the orbits of the massive binary stars (i.e., massless test particle).  Through the many exoplanet discoveries made over the last few decades, these assumptions have not held up to reality.  It is true that we may expect circumstellar planets to begin with on nearly circular and coplanar orbits relative to the host binary, but disk interactions or planet-planet interactions over the lifetime of the system can perturb the discovered planets away from our ideal assumption.  

Quarles et al. (2020) provide a somewhat different approach.  Instead of using a fitting function based on some physical parameters (e.g., stellar dynamical mass ratio $\mu$ and binary eccentricity $e_{bin}$), they use an interpolation scheme that is built upon a much larger set of N-body simulations.  In these simulations, all three bodies interact with one another gravitationally and more accurately represent what we expect to observe.  This Jupyter notebook will show you how to utilize the results of Quarles et al. (2020) using $\alpha$ Centauri as an example.

# Physical observables
For wide binary systems, one can usually obtain an estimate of the mass quotient $q=M_B/M_A$, binary orbital period $P_{bin}$, and the binary eccentricity $e_{bin}$ from modeling the observed photometric data (i.e., light curve) and radial velocity measurements (RV curve) together.  More intense modeling can help determine the individual masses of the stars, but we can convert between the observational mass quotient $q$ to the dynamical mass ratio $\mu$ through the following:

$\mu = \frac{M_B}{M_A+M_B} = \frac{q}{1+q}$

or

$q = \frac{\mu}{1-\mu}$

# $\alpha$ Centauri 
From Pourbaix & Boffin (2016), we have measurements of the $\alpha$ Centauri AB binary for $M_A=1.133 \pm 0.005$, $M_B=0.972 \pm 0.005$, $e_{bin} = 0.524 \pm 0.001$, and $P_{bin} = 79.91 \pm 0.013$ years.  Now to the preliminaries of the code:

In [12]:
import numpy as np  #We need numpy for the convenience function genfromtxt
from scipy.interpolate import griddata,interp2d  #We need this to map our data to a grid and interpolate between grid points

def get_stability_limit(f,mu,e_bin):
    return f(mu,e_bin)[0]

The import statements above include the standard packages from numpy and scipy.  The function **get_stability_limit** is something for convenience so that we may generalize later and is passed a set of parameters.  The interpolation function $f$ that we create using scipy and the binary parameters.  Next, let's create the interpolation function for a coplanar planet ($i_p = 0^\circ$) and determine the stability limit for a planet orbiting star A.

In [13]:
ip = 0
data = np.genfromtxt("a_crit_Incl[%i].txt" % ip,delimiter=',',comments='#')  #The data contained in this repository

X = data[:,0] #mu
Y = data[:,1] #e_bin
Z = data[:,2] #a_c/a_bin

xi = np.concatenate(([0.001],np.arange(0.01,1,0.01),[0.999]))
yi = np.arange(0,0.81,0.01)
zi = griddata((X,Y),Z,(xi[None,:],yi[:,None]),method = 'linear',fill_value=0)  #make the grid

f = interp2d(xi, yi, zi, kind='linear') # make the 2d interpolation

In [16]:
M_tot = 0.972 + 1.133 # Total star mass in M_sun
mu = 0.972/(0.972 + 1.133)  #converting from M_A, M_B --> mu  Pourbaix & Boffin (2016)
e_bin = 0.524
P_bin = 79.91
a_bin = (P_bin**2*M_tot)**(1./3)

print("a_c = %1.3f AU" % (get_stability_limit(f,mu,e_bin)*a_bin))


a_c = 2.773 AU


Let's compare the critical semimajor axis $a_c$ to the long-term (10 Gyr) simulations from Quarles, Lissauer, and Kaib (2018).  They determine the stability limit for a initially ciruclar planet orbiting star A at around 2.7 AU, where this can be extended to 2.77 AU if the planet begins with an appropriate forced eccentricity.